# Combinación de ambos modelos

En este notebook vamos a recrear el proceso completo con los dos modelos que hemos entrenado, desde cargar una imagen inicial hasta la obtención del archivo MIDI final.

## Cargamos los modelos

In [1]:
from model.extender_midi.model import MelodIAModel, Utilities
from model.foto_a_midi.main import FotoAMIDI

# Cargamos los modelos
MODEL_PARAMS_PATH = "pretrained_models/modelo_params_midi.h5"
MODEL_MIDI_EXTENSION_PATH = "pretrained_models/modelo_extension_midi.keras"

modelo_params = FotoAMIDI(modelo_ruta=MODEL_PARAMS_PATH, usar_clip=True)
modelo_params.load_modelo()
modelo_extender_midi = MelodIAModel.load_model(MODEL_MIDI_EXTENSION_PATH)

2025-05-18 17:33:06.791519: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-18 17:33:06.802055: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747582386.815879   76692 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747582386.820181   76692 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747582386.829657   76692 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Modelo cargado desde: pretrained_models/modelo_extension_midi.keras


## Modelo 1: Foto a MIDI

## Modelo 2: Extendión del archivo MIDI generado

In [ ]:
import pretty_midi

df = Utilities.midi_path_to_df(midi_path)
notes = Utilities.midi_to_numpy(pm)
notes_sequence = notes[:25] # primeras 25 notas
notes = MelodIAModel.normalize(notes_sequence) 

Generamos las nuevas notas con el modelo

In [ ]:
# Se pueden cambiar el número de predicciones y la temperatura
generated_notes = MelodIAModel.predict_sequence(model, notes, num_predictions=25, temperature=0.8)

generated_notes = MelodIAModel.desnormalize(generated_notes)
generado = Utilities.notes_to_midi(generated_notes)

Unimos el fragmento MIDI original con el generado

In [ ]:
original = Utilities.notes_to_midi(notes_sequence)
combined = Utilities.concatenate_pretty_midi(original, generado)

Lo visualizamos

In [ ]:
Utilities.plot_piano_roll(combined, 40, 100)

También se puede escuchar

In [ ]:
Utilities.display_audio(combined)

Por último lo guardamos

In [ ]:
combined.write("midi_final.mid")